# Deep Learning

Because of the static multiple dispatch paradigm layed out in [Multiple Dispatch.ipynb](./Multiple%20Dispatch.ipynb), we need to first include the primitive operations for the device(s) we are inteding on using such that the algorithms (and datastructures) we later include for deep learning can use them.

In [1]:
#include <backprop_tools/operations/cpu.h>

In [2]:
#include <backprop_tools/nn/layers/dense/operations_cpu.h>

We set up the environment as described in [Containers.ipynb](./Containers.ipynb):

In [3]:
namespace bpt = backprop_tools;
using DEVICE = bpt::devices::DefaultCPU;
using T = float;
using TI = typename DEVICE::index_t;
DEVICE device;
TI seed = 1;
auto rng = bpt::random::default_engine(DEVICE::SPEC::RANDOM(), seed);

As justified by our analysis of the reinforcement learnign for continuous control landscape (in the [paper](https://arxiv.org/abs/2306.03530)) in the beginning **BackpropTools** only supports fully connected neural networks. But we are planning on adding more architectures (especially recurrent neural networks) in the future.

We can instantiate a simple layer by first defining its hyperparameters (which are compile-time `constexpr` and types):

In [4]:
constexpr TI INPUT_DIM = 5;
constexpr TI OUTPUT_DIM = 5;
constexpr auto ACTIVATION_FUNCTION = bpt::nn::activation_functions::RELU;
using PARAMETER_TYPE = bpt::nn::parameters::Plain;

We will explain the role of the `PARAMETER_TYPE` later on. 

These hyperparameters and other options are combined into a specification type such that it is easier to pass it around and such that we don't need to write out all hyperparameters and options as template parameters when a function takes the datastructure as an argument:

In [5]:
using LAYER_SPEC = bpt::nn::layers::dense::Specification<T, TI, INPUT_DIM, OUTPUT_DIM, ACTIVATION_FUNCTION, PARAMETER_TYPE>;

Using this specification we can declare an actual layer:

In [6]:
bpt::nn::layers::dense::Layer<LAYER_SPEC> layer;

A fully connected neural network consists of layers each implementing: $$y = f(Wx + b)$$ where $x$ is the input (external or from the previous layer), $W$ and $b$ are the weight matrix and biases respectively and $f$ is an element-wise non-linear function. Hence the data structure of a layer should contain at least $W$ and $b$. Because these parameters are containers they need to be allocated:

In [7]:
bpt::malloc(device, layer);

Now that the memory is allocated we need to initialize it (because it may contain arbitrary values). We use the standard [Kaiming](https://pytorch.org/docs/stable/nn.init.html?highlight=kaiming#torch.nn.init.kaiming_normal_) initialization scheme:

In [8]:
bpt::init_kaiming(device, layer, rng);

We can print $W$ and $b$:

In [9]:
bpt::print(device, layer.weights.parameters)

   -0.329563     0.228620    -0.036984     0.029308    -0.251371 
    0.159981     0.160368     0.388801    -0.104199     0.017367 
   -0.416291    -0.399396     0.026565     0.153081    -0.440328 
   -0.387428    -0.073803     0.167055     0.079583     0.384994 
    0.024086    -0.364958     0.137669    -0.075132     0.179950 


In [10]:
bpt::print(device, layer.biases.parameters)

   -0.447207    -0.405136     0.296024    -0.104276     0.309621 


Now that the layer is initialized we can run inference using a random input. We first declare and allocate input and output matrices and then randomly initialize the input:

In [11]:
constexpr TI BATCH_SIZE = 1;
bpt::MatrixDynamic<bpt::matrix::Specification<T, TI, BATCH_SIZE, INPUT_DIM>> input;
bpt::MatrixDynamic<bpt::matrix::Specification<T, TI, BATCH_SIZE, INPUT_DIM>> output;
bpt::malloc(device, input);
bpt::malloc(device, output);
bpt::randn(device, input, rng);
bpt::print(device, input);

    0.175199    -0.863064     1.316539     0.942564    -0.589718 


Now we can evaluate output of the layer:

In [12]:
bpt::evaluate(device, layer, input, output);
bpt::print(device, output);

    0.000000     0.000000     1.006727     0.000000     0.633133 


Now we are revisiting the `PARAMETER_TYPE` template argument. 
For inference storing $W$ and $b$ is sufficient but for training we at least need to also store the gradient of the loss $L$ wrt. $W$ and $b$: $\frac{\mathrm{d}L}{\mathrm{d}W}$ and $\frac{\mathrm{d}L}{\mathrm{d}b}$. Because depending on the optimizer type we might need to store more information per parameter (like the first and second-order moment in the case of [Adam](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#Adam)), we abstract the storage for the weights and biases using a `PARAMETER_TYPE` that can e.b. be `Plain`, `Gradient`, `Adam` or any other type extended by the user. For this illustration we are using `Gradient`:

In [13]:
using PARAMETER_TYPE_2 = bpt::nn::parameters::Gradient;
using LAYER_2_SPEC = bpt::nn::layers::dense::Specification<T, TI, INPUT_DIM, OUTPUT_DIM, ACTIVATION_FUNCTION, PARAMETER_TYPE_2>;
bpt::nn::layers::dense::LayerBackwardGradient<LAYER_2_SPEC> layer_2;
bpt::malloc(device, layer_2);
bpt::copy(device, device, layer_2, layer);
bpt::zero_gradient(device, layer_2);

Note that we now use the `bpt::nn::layers::dense::LayerBackwardGradient` datastructure which is supported by the functions implementing the backpropagation algorithm. Additionally, similar to PyTorch we are setting the gradient to zero because it is accumulated with subsequent backward passes.

Now we can backpropagate the derivative of the loss wrt. the `output` to calculate the derivative of the loss wrt. the `input`. Hence the derivative of the loss wrt. the `output`: `d_output` is actually an input to the `bpt::backward` operator. The operator also accumulates the derivative of the loss wrt. the weights and biases in the layer. We first allocate containers for `d_input` and `d_output` and randomly set `d_output` (a hypothetical gradient of the input of some upstream layers)

In [14]:
bpt::MatrixDynamic<bpt::matrix::Specification<T, TI, BATCH_SIZE, OUTPUT_DIM>> d_output;
bpt::MatrixDynamic<bpt::matrix::Specification<T, TI, BATCH_SIZE, INPUT_DIM>> d_input;
bpt::malloc(device, d_input);
bpt::malloc(device, d_output);
bpt::randn(device, d_output, rng);

Now we execute the backpropagation and display the gradient of the loss wrt. the input:

In [15]:
bpt::backward(device, layer_2, input, d_output, d_input);
bpt::print(device, d_input);

   -0.099916     0.307348    -0.325702     0.127334    -0.196570 


This also accumulates the gradient in the weights and biases:

In [16]:
bpt::print(device, layer_2.weights.gradient);

    0.000000     0.000000     0.000000     0.000000     0.000000 
   -0.081795     0.402941    -0.614657    -0.440058     0.275324 
    0.000000     0.000000     0.000000     0.000000     0.000000 
    0.000000     0.000000     0.000000     0.000000     0.000000 
   -0.183485     0.903885    -1.378809    -0.987145     0.617611 


In [17]:
bpt::print(device, layer_2.biases.gradient);

    0.000000    -0.466873     0.000000     0.000000    -1.047298 


In [18]:
bpt::free(device, layer);
bpt::free(device, layer_2);
bpt::free(device, input);
bpt::free(device, output);
bpt::free(device, d_input);
bpt::free(device, d_output);

Until now we showed the behavior of a single, fully-connected layer. **BackpropTools** contains an [Multilayer Perceptron (MLP)](https://en.wikipedia.org/wiki/Multilayer_perceptron) that conveniently integrates an arbitrary number of layers into a single data structure with algorithms to perform forward passes and backpropagation across the whole model.

